You have to work on the [Dogs adoptions](https://drive.google.com/file/d/1wQsA0oB6wwYlnkvvcyBCmLk7QmgVWNax/view?usp=sharing) dataset. 

It contains three files:
*  `dogs.csv`, shortly *dogs*
*  `dogTravel.csv`, shortly *travels*
*  `NST-EST2021-POP.csv`

### Notes

1.    It is mandatory to use GitHub for developing the project.
1.    The project must be a jupyter notebook.
1.    There is no restriction on the libraries that can be used, nor on the Python version.
1.    All questions on the project **must** be asked in a public channel on [Zulip](https://focs.zulipchat.com).
1.    At most 3 students can be in each group. You must create the groups by yourself.
1.    You do not have to send me the project *before* the discussion.

### 0.1 Importing files

In [ ]:
# Importing Pandas
import pandas as pd

# Opening dogs.csv and checking columns
with open("dogs.csv", "r") as dogs_file:
    headers = dogs_file.readline()
    print(headers)

In [ ]:
# Creating 'dogs' df 
dogs = pd.read_csv("dogs.csv", sep=',', doublequote='"', low_memory=False)

# Checking the head
dogs.head()

### 0.2.1 Dogs df cleaning

In [ ]:
# For cleaning purpose, from this time on multiple dataframes are created
# tmp_dog_full is the original dataframe. His shape is shown below.
tmp_dog_full = pd.read_csv("dogs.csv", sep=',', doublequote='"', low_memory=False, encoding='utf-8')
print(f'tmp_dog_full shape: {tmp_dog_full.shape}')

# A new column called 'ok' is created, which is set to the opposite of whether the 'contact_state' column
# contains numeric value or not. It checks which lines are ok and what needs to be managed in a different way, using contact_state as watermark
tmp_dog_full['ok'] = ~tmp_dog_full.contact_state.str.isnumeric()

# Makes all the column names lowercase and replaces dots with underscores in them
tmp_dog_full.columns = [col.lower().replace(".", "_") for col in tmp_dog_full.columns]

# A new dataframe, called tmp_dog_ok is created. It contains the rows
# where 'ok' is True.
tmp_dog_ok = tmp_dog_full[tmp_dog_full.ok == True]
print('tmp_dog_ok:')
display(tmp_dog_ok.head(5))

# Checks that all rows are ok, and prints the number of unique contact_state.
print('check all rows are ok')
print(len(tmp_dog_ok.contact_state.unique()))
tmp_dog_ok.contact_state.unique()

In [ ]:
# A new df, tmp_dog_not_ok, is created. It only contains the rows from the
# original dataframe tmp_dog_full where the column 'ok' is False.
# This is the complement of the dataframe tmp_dog_ok created previously.
tmp_dog_not_ok = tmp_dog_full[tmp_dog_full.ok == False]
print('tmp_dog_not_ok')
display(tmp_dog_not_ok.head(5))

In [ ]:
pd.set_option('display.max_colwidth', 100) #50

# Managing "not ok" dataframe: split name column and shift the others
# Let's see what the dataframe looks like before the cleaning.
print('before')
display(tmp_dog_not_ok.head(1))

# tmp_dog_not_ok_fixed is created, with the same columns and indices as the original dataframe
tmp_dog_not_ok_fixed = pd.DataFrame(columns=tmp_dog_not_ok.columns, index=tmp_dog_not_ok.index)

# Copies the columns from 0 to 24 and from 26 to the end of the original datafame
# to the new dataframe, but dropping the column 'accessed'.
tmp_dog_not_ok_fixed.iloc[:, 0:24] =  tmp_dog_not_ok.iloc[:, 0:24].copy()
tmp_dog_not_ok_fixed.iloc[:, 26:] =  tmp_dog_not_ok.iloc[:, 25:].drop('accessed', axis = 1).copy()

# Taking the 24th column of the original dataframe and splitting it into
# two new columns, 'name' and 'status'.
tmp_dog_not_ok.iloc[: , 24]
tmp_dog_not_ok_fixed.name = tmp_dog_not_ok.name.apply(lambda x : x.split('\",')[0])
tmp_dog_not_ok_fixed.status = tmp_dog_not_ok.name.apply(lambda x : x.split('\",')[1].strip('"'))
print('after')
tmp_dog_not_ok_fixed.head()


In [ ]:
# Combining the two dataframes, 'tmp_dog_ok' and 'tmp_dog_not_ok_fixed' into 'dogs'.
# It concatenates the two dataframes vertically.
print('tmp_dog_ok shape:', tmp_dog_ok.shape)
print('tmp_dog_not_ok shape:', tmp_dog_not_ok.shape)
dogs = pd.concat([tmp_dog_ok, tmp_dog_not_ok_fixed])
print('dogs shape:', dogs.shape)

# Temporary dataframes are deleted.
del tmp_dog_full
del tmp_dog_not_ok
del tmp_dog_not_ok_fixed
del tmp_dog_ok

# Makes all column names lowercase and replaces dots with underscores
dogs.columns = [col.lower().replace(".", "_") for col in dogs.columns]

# Dropping the 'ok' column
dogs.drop('ok', axis=1, inplace=True)
dogs.columns

### 0.2.2 Travels dataset cleaning

In [ ]:
# The travels dataset is stored into the temporary 'tmp_travels' dataframe
tmp_travels = pd.read_csv("dogTravel.csv", sep=',', doublequote='"', low_memory=False).drop('index', axis=1)

# Data exploration
display(tmp_travels.head())
display(tmp_travels.contact_state.unique())
display(tmp_travels[tmp_travels.contact_state == '17325'].id.unique())

# The variable 'anomalies' is created, which contains the unique 'id' values
# of the rows where the 'contact_state' is '17325'
anomalies = tmp_travels[tmp_travels.contact_state == '17325'].id.unique()

# Changing the 'contact_state' value of all the rows where the 'id' is equal 
# to the anomalies declared before.
tmp_travels.loc[tmp_travels.id == anomalies[0], 'contact_state'] = 'PA'
tmp_travels.loc[tmp_travels.id == anomalies[1], 'contact_state'] = 'PA'
display(tmp_travels[tmp_travels.id.isin(anomalies)])
display(tmp_travels.contact_state.unique())

# Storing the tmp_travels with the fixed values into the original one, 'travels'
travels = tmp_travels.copy()
del tmp_travels

### 0.2.3 States df cleaning

In [ ]:
# Storing the states dataset into tmp_states
tmp_states = pd.read_csv("NST-EST2021-POP.csv", header=None, names=["state", "population"], sep=',', low_memory=False)
tmp_states.head()

In [ ]:
# Replacing all occurrences of '.' in the 'population' column with an empty string
tmp_states.population = tmp_states.population.str.replace('.', '', regex=False).astype(int)

# Creating the original 'states' dataframe
states = tmp_states.copy()
del tmp_states
states.head()

### 1. Extract all dogs with status that is not *adoptable*

In [ ]:
pd.set_option('display.max_rows', 10) #50

print(dogs[dogs.status != 'adoptable'].shape)
not_adoptable_dogs = dogs[dogs.status != 'adoptable']

display(not_adoptable_dogs)

### 2. For each (primary) breed, determine the number of dogs

In [ ]:
dogs['breed_primary'].value_counts()

### 3. For each (primary) breed, determine the ratio between the number of dogs of `Mixed Breed` and those not of Mixed Breed. Hint: look at the `secondary_breed`.

In [ ]:
# Printing the distinct breeds
print('distinct breed: ', len(dogs.breed_primary.unique()))

# Creating a new dataframe that groups the 'dogs' dataframe by the 'breed_primary' 
# column, counting the number of dogs for each breed and returning 
# the result sorted by the number of dogs
breeds = dogs.groupby('breed_primary', as_index=False).count()[['breed_primary','id']].rename({'id': 'number_of_dogs'}, axis=1).sort_values(by='number_of_dogs', ascending=False)
breeds

In [ ]:
# Creating a new dataframe sec_breeds that contains the count of mixed dogs 
# for each primary breed, by first filtering the 'dogs' dataframe to include only 
# the rows with not null values in the 'breed_secondary' column, then grouping 
# the filtered dataframe by 'breed_primary' and counting the number of dogs for each primary breed 
sec_breeds = dogs[dogs.breed_secondary.notnull()]
sec_breeds = sec_breeds.groupby('breed_primary', as_index=False).count()[['breed_primary','id']].rename({'id': 'number_of_dogs'}, axis=1)
sec_breeds.head()

In [ ]:
# Merging 'breeds' and 'sec_breeds' with a left-join into mix_breeds
mix_breeds = breeds.merge(sec_breeds, on='breed_primary', how='left', suffixes=('_tot','_mixed'))
mix_breeds['number_of_dogs_mixed'] = mix_breeds['number_of_dogs_mixed'].fillna(0)

# Calculating the ratio
mix_breeds['mixed_ratio_perc'] = mix_breeds.apply(lambda x : round(x.number_of_dogs_mixed/x.number_of_dogs_tot, 2)*100, axis=1)
mix_breeds['pure_ratio_perc'] = mix_breeds.apply(lambda x : 100 - x.mixed_ratio_perc, axis=1)
mix_breeds

### 4. For each (primary) breed, determine the earliest and the latest `posted` timestamp.

In [ ]:
# Formatting the 'posted' column
dogs['posted'] = pd.to_datetime(dogs['posted'], errors="coerce")

# Creating the df with earliest and latest 'posted' timestamps
earliest_latest_timestamp = dogs.groupby('breed_primary', as_index=False).aggregate({'posted':[min, max]})

earliest_latest_timestamp

### 5. For each state, compute the sex imbalance, that is the difference between male and female dogs. In which state this imbalance is largest?

In [ ]:
malefemale = dogs[['contact_state', 'contact_city', 'contact_zip', 'contact_country', 'sex']].copy()

# creating a new dataframe 'malefemale' and 'malefemale_imbalance' that shows the imbalance of male to female dogs by state
malefemale['imbalance'] = malefemale.sex.apply(lambda x : 1 if x.upper() == 'MALE' else -1)
malefemale_imbalance = malefemale.groupby('contact_state', as_index=False).sum('imbalance')[['contact_state', 'imbalance']]
malefemale_imbalance.iloc[[malefemale_imbalance.imbalance.idxmin(), malefemale_imbalance.imbalance.idxmax()]]

### 6. For each pair (age, size), determine the average duration of the stay and the average cost of stay.

In [ ]:
# Converting the 'stay_duration' and 'stay_cost' to integer and float
dogs.stay_duration = dogs.stay_duration.astype(int)
dogs.stay_cost = dogs.stay_cost.astype(float)

# Grouping the dogs by age and size, calculating the mean of the stay_duration and stay_cost
stay = dogs.groupby(['age', 'size'], as_index=False).agg({'stay_duration' : 'mean', 'stay_cost' : 'mean'})

# Rounding the values to 2 decimal places
stay.stay_duration = stay.stay_duration.apply(lambda x : round(x, 2))
stay.stay_cost = stay.stay_cost.apply(lambda x : round(x, 2))
stay

### 7. Find the dogs involved in at least 3 travels. Also list the breed of those dogs.

In [ ]:
# Counting the number of occurrences of each unique dog that traveled at least 3 times
many_travels = travels[['id', 'contact_state']].groupby('id', as_index=False).count().rename({'contact_state':'travels'}, axis=1)
many_travels = many_travels[many_travels.travels > 2]
many_travels

In [ ]:
# Just a check
travels[travels.id == 46042569]

In [ ]:
# Merging many_travels and 'dogs' on the 'id' column
breed_travels = many_travels.merge(dogs[['id', 'breed_primary']], left_on='id', right_on='id')

# Sorting by the 'travels' column
breed_travels.sort_values('travels', ascending=False)

### 8. Fix the `travels` table so that the correct state is computed from  the `manual` and the `found` fields. If `manual` is not missing, then it overrides what is stored in `found`.

In [ ]:
# Creating a copy
exercise_8 = travels.copy()

# If the value in the 'manual' column is null, then it assigns the value of the 'found' column of that 
# row, otherwise it assigns the value of the 'manual' column of that row.
exercise_8.found = exercise_8.apply(lambda x : x['found'] if pd.isnull(x['manual']) else x['manual'] ,axis=1)
exercise_8

### 9. For each state, compute the ratio between the number of travels and the population.

In [ ]:
# Charging a decoding table
abbreviations = pd.read_csv("abbreviations.csv", sep=',', quotechar='"')
print(abbreviations.shape)
abbreviations

In [ ]:
## Checking for anomalies before the merge
anomalies = [s for s in states['state'].unique() if s not in abbreviations.state.unique()]
print(f"anomalies number in states df: {len(anomalies)}")

anomalies = [s for s in travels['contact_state'].unique() if s not in abbreviations.code.unique()]
print(f"anomalies number in travels df: {len(anomalies)}")

print('next state is missing in file of population: located in Canada')
travels[travels['contact_state'].isin(anomalies)]


In [ ]:
# Counting the number of occurrences of each unique 'contact_state' value, returning the count of 
# travels grouped by contact_state
travels_by_states = travels.groupby('contact_state', as_index=False).count()[['contact_state', 'id']].rename({'id' : 'travels', 'contact_state': 'code'}, axis=1)
travels_by_states

In [ ]:
# Matching state name to state code
states_travels = states.merge(abbreviations[['state', 'code']], on='state')
states_travels

In [ ]:
# Filling missing values and computing the ratio
states_travels = states_travels.merge(travels_by_states, on='code', how='left').fillna(0)
states_travels['travels_per_people'] = states_travels.apply(lambda x : x['travels']/x['population'], axis=1)
states_travels

### 10. For each dog, compute the number of days from the `posted` day to the day of last access.

In [ ]:
# Creating a df copy for this exercise
exercise_10 = dogs[['id', 'name', 'posted', 'accessed']].copy()

# Computing the number of days from the 'posted' day to the day of last access, assuming it's 'accessed' column
# The value is stored in 'days_delay' column
exercise_10['posted'] = pd.to_datetime(pd.to_datetime(exercise_10['posted']).dt.date)
exercise_10['accessed'] = pd.to_datetime(exercise_10['accessed'])
exercise_10['days_delay'] = (exercise_10['accessed'].dt.date - exercise_10['posted'].dt.date).dt.days

# Printing the result
exercise_10

### 11. Partition the dogs according to the number of weeks from the `posted` day to the day of last access.

In [ ]:
# Creating a df copy for this exercise
exercise_11 = exercise_10

# Creating a new column, 'weeks', that stores the number of weeks from the posted day to the day of last access
exercise_11["weeks"] = round(exercise_11["days_delay"] // 7,0).astype(int)

# Grouping the dogs in different partitions, based on 'weeks' value
partitioned_dogs = exercise_11.groupby("weeks").count()[['id']].rename({'id': 'number_of_dogs'}, axis=1)

# Printing them
partitioned_dogs

### 12. Find for duplicates in the `dogs` dataset. Two records are duplicates if they have (1) same breeds and sex, and (2) they share at least 90% of the words in the description field. Extra points if you find and implement a more refined for determining if two rows are duplicates.

In [ ]:
# Lowercase, remove punctuation, tokenize, lemmatization
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import re
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('stopwords')

lemmatizer = WordNetLemmatizer()
stop = stopwords.words('english')
stop.extend(['dog', 'dogs', '-', 'old'])

In [ ]:
# First step: remove dogs with no description
descripted_dogs = dogs[dogs['description'].notnull()]
print(f'{len(descripted_dogs)} have description')

In [ ]:
# Second step: find dog by equal 'breed_primary', 'sex', 'description'
dupdog1= descripted_dogs[descripted_dogs[['breed_primary', 'sex', 'description']].duplicated(keep='first')][['id','breed_primary','sex', 'description']] 
dupdog2= descripted_dogs[descripted_dogs[['breed_primary', 'sex', 'description']].duplicated(keep='last')][['id','breed_primary','sex', 'description']]
duplicated_dogs = pd.concat([dupdog1, dupdog2])
duplicated_dogs = duplicated_dogs.drop_duplicates()
duplicated_dogs_id = list(duplicated_dogs['id'])

print(f'found {len(duplicated_dogs_id)} duplicated dogs on {descripted_dogs.shape[0]} dogs with description')

duplicated_dogs.sort_values(by=['sex','breed_primary'])

In [ ]:
# Third step: catching remaining duplicates
print(f'{len(descripted_dogs)} - {len(duplicated_dogs)}' )

# Removing duplicated records from descripted_dogs and then storing
# the dataframe into dogs_12
idx = descripted_dogs.index[descripted_dogs.isin(duplicated_dogs).any(1)]
descripted_dogs = descripted_dogs.drop(idx)
dogs_12 = descripted_dogs

print(f'remain {len(dogs_12)} on {descripted_dogs.shape[0]} dogs')

### SECOND OPTION ###
# substraction_df = pd.merge(descripted_dogs, duplicated_dogs, how='outer', indicator='Duplicated')
# dogs_12 = substraction_df[substraction_df["Duplicated" == 'left_only']]


In [ ]:
# Filtering the dataframe to exclude NaN records from the 'description' column
dogs_12['lemm_description'] = dogs_12.description.str.lower().str.replace('[^a-zA-Z0-9 \w+\.\w+@\w+\.\w \w+@\w+\.\w www.\w+\.\w]',' ', regex=True)    
dogs_12['lemm_description'] = dogs_12['lemm_description'].str.lower().str.replace('(\w)(\. )',r'\1 ', regex=True).str.strip('.')  
dogs_12['lemm_description'] = dogs_12['lemm_description'].apply(lambda x: ' '.join([lemmatizer.lemmatize(word) for word in x.split() if word not in stop])) 

In [ ]:
from collections import Counter
pd.set_option('display.max_colwidth', 500) #50

# Creating the 'cleaned_description' column, applying several operations on the lemm_description column in dogs_12
dogs_12['cleaned_description'] = dogs_12.lemm_description.str.replace('(\w+)? ?(\d+) (\w+)',r'\1\2\3', regex=True)
dogs_12['cleaned_description'] = dogs_12.cleaned_description.str.replace(' \w ',' ', regex=True)
dogs_12['cleaned_description'] = dogs_12.cleaned_description.str.replace('\s+',' ', regex=True)

# Pruning the rows where 'cleaned_description' is null
print(f'rows before pruning: {len(dogs_12)}')
dogs_12 = dogs_12[dogs_12['cleaned_description'].notnull()]
print(f'rows after pruning: {len(dogs_12)}')

# Creating the description_counter and description_dictionary columns, with the frequency
# of words and the keys of description_counter
dogs_12['description_counter'] = dogs_12['cleaned_description'].apply(lambda x: dict(Counter(x.split()))) 
dogs_12['description_dictionary'] = dogs_12['description_counter'].apply(lambda x: sorted(set(x.keys()))) 

In [ ]:
# Creating clusters by the 'breed_primary' and 'sex' columns, counting 
# the number of dogs in each group.
print(f'dogs before clustering: {dogs_12.shape[0]}')
dogs_clusters = dogs_12.groupby(['breed_primary', 'sex'])[['id']].count().reset_index().rename(columns={'id':'counts'})
print(f'dogs after clustering: {dogs_clusters.counts.sum()}')
print(f'dogs clusters: {dogs_clusters.shape}')
dogs_clusters

In [ ]:
# Creating an empty list where the duplicates will be stored
duplicated_couples = []
counter = 0

# Filtering one sex at a time to optimize calculations
for sex in ['Male', 'Female']:
    clusters_by_sex = dogs_clusters[dogs_clusters['sex'] == sex][['breed_primary','counts']]
    dogs_by_sex = dogs_12[dogs_12['sex'] == sex]
    cluster_size = clusters_by_sex.shape[0]
    print(f'sex: {sex}')
    cluster_number = 0
    
    # Analyzing one cluster at a time
    for breed_primary, counts in clusters_by_sex.values:
        
        cluster_number = cluster_number + 1 
        print(f'processing cluster number: {cluster_number} of {cluster_size}--> {breed_primary} ({counts})')
        
        
        this_cluster = dogs_by_sex[dogs_by_sex['breed_primary']==breed_primary]
        duplicated_id_already_found = []

        # Comparing each record with the others in the cluster
        for i in range(0, counts-1):

            first_dog = this_cluster.iloc[i]
            desc1 = first_dog['cleaned_description']
            set1 = first_dog['description_dictionary']

            # Starts another nested loop starting from the next dog and iterating over all the remaining dogs in the cluster
            for j in range(i+1, counts):
                counter = counter + 1
                second_dog = this_cluster.iloc[j]
                desc2 = second_dog['cleaned_description']
                set2 = second_dog['description_dictionary']

                # it calculates the overlap ratio by taking the length of the union of 
                # the 'description_dictionary' of the two dogs and dividing it by the length of their intersection.
                union = len(set1 | set2)
                intersect = len(set1 & set2)
                overlap_ratio = intersect / union

                duplicated_couples.append({'sex': sex, 'breed_primary':breed_primary, 'first':first_dog['id'], 'second':second_dog['id'], 'overlap_ratio':overlap_ratio})

# Saving the duplicates to an external CSV for quick checks
df = pd.DataFrame(duplicated_couples)
df.to_csv(f'overlap_optimized.csv', index=False, sep=',', encoding='utf-8')  

print(counter)

In [ ]:
# Calculating the stopwerds percentage on all descriptions
all_description_words = dogs_12.description.apply(lambda x : len(str(x).split())).sum()
all_cleaned_words = dogs_12.cleaned_description.apply(lambda x : len(str(x).split())).sum()
ratio_cleaned_words = all_cleaned_words/all_description_words

print(f"""{ratio_cleaned_words}% of words are stopwords""")

In [ ]:
df = pd.read_csv(f'duplicates_full_optimized.csv', sep=',', encoding='utf-8')
df.shape

In [ ]:
starting_threshold = 0.9
estimate_threshold = starting_threshold * ratio_cleaned_words
print(estimate_threshold)
df['overlap_class'] = df.overlap_ratio.apply(lambda x: x if x == 1 else x//0.1/10)

In [ ]:
df_dup = df.copy()
df_dup = df_dup[df_dup.overlap_class > estimate_threshold]
df_dup.describe()

In [ ]:
print(df_dup.shape)
df_dup_single = pd.concat([df_dup.rename({'first':'id'}, axis=1)[['id','sex','breed_primary', 'overlap_class']] 
                              , df_dup.rename({'second':'id'}, axis=1)[['id','sex','breed_primary', 'overlap_class']]]).drop_duplicates()
print(df_dup_single.shape)
df_dup_single.head()

In [ ]:
df_dup_single.overlap_class.plot.hist()

In [ ]:
#some random check for those first dog that have the same description
print('it seems that most of descriptions that are identical, are ads')
dogs[dogs.id.isin(duplicated_dogs[duplicated_dogs.description == duplicated_dogs.iloc[28].description]['id'])][['breed_primary','sex','name','description']].sample(5)


In [ ]:
# print result
print(len(duplicated_dogs_id))
duplicated_dogs_id.extend(list(df_dup_single['id'].values))
print(len(set(duplicated_dogs_id)))
result = dogs[dogs['id'].isin(set(duplicated_dogs_id))]
result